In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

In [2]:
data = pd.read_csv("train.csv")
data

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056
878045,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,-122.447364,37.731948
878046,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


In [4]:
data.duplicated().sum()

2323

In [5]:
data.drop_duplicates(subset=None, keep="first", inplace=True)

In [6]:
data['Category'].value_counts(normalize=True)

LARCENY/THEFT                  0.199058
OTHER OFFENSES                 0.143835
NON-CRIMINAL                   0.104959
ASSAULT                        0.087716
DRUG/NARCOTIC                  0.061571
VEHICLE THEFT                  0.061327
VANDALISM                      0.050907
WARRANTS                       0.048126
BURGLARY                       0.041794
SUSPICIOUS OCC                 0.035849
MISSING PERSON                 0.029312
ROBBERY                        0.026250
FRAUD                          0.018998
FORGERY/COUNTERFEITING         0.012095
SECONDARY CODES                0.011395
WEAPON LAWS                    0.009763
PROSTITUTION                   0.008503
TRESPASS                       0.008356
STOLEN PROPERTY                0.005181
SEX OFFENSES FORCIBLE          0.005002
DISORDERLY CONDUCT             0.004925
DRUNKENNESS                    0.004884
RECOVERED VEHICLE              0.003576
KIDNAPPING                     0.002672
DRIVING UNDER THE INFLUENCE    0.002590


In [7]:
data['Descript'].value_counts(normalize=True)

GRAND THEFT FROM LOCKED AUTO                            0.068246
LOST PROPERTY                                           0.035968
BATTERY                                                 0.031308
STOLEN AUTOMOBILE                                       0.030679
DRIVERS LICENSE, SUSPENDED OR REVOKED                   0.030629
                                                          ...   
EMBEZZLEMENT, GRAND THEFT BY PROPERTY CARRIER           0.000001
ASSAULT, AGGRAVATED, ON POLICE OFFICER, W/ SEMI AUTO    0.000001
DISTURBANCE OF NON-RELIGIOUS, NON-POLITICAL ASSEMBLY    0.000001
VIOLATION OF CALIF UNEMPLOYMENT INSURANCE ACT           0.000001
ATTEMPTED THEFT PHONE BOOTH                             0.000001
Name: Descript, Length: 879, dtype: float64

In [8]:
data['DayOfWeek'].value_counts(normalize=True)

Friday       0.152298
Wednesday    0.147111
Saturday     0.144416
Thursday     0.142461
Tuesday      0.142285
Monday       0.138517
Sunday       0.132911
Name: DayOfWeek, dtype: float64

In [9]:
data['PdDistrict'].value_counts(normalize=True)

SOUTHERN      0.178741
MISSION       0.136713
NORTHERN      0.120020
BAYVIEW       0.101742
CENTRAL       0.097367
TENDERLOIN    0.093228
INGLESIDE     0.089893
TARAVAL       0.074681
PARK          0.056137
RICHMOND      0.051477
Name: PdDistrict, dtype: float64

In [10]:
data['Resolution'].value_counts(normalize=True)

NONE                                      0.599642
ARREST, BOOKED                            0.235431
ARREST, CITED                             0.087735
LOCATED                                   0.019286
PSYCHOPATHIC CASE                         0.016591
UNFOUNDED                                 0.010936
JUVENILE BOOKED                           0.006354
COMPLAINANT REFUSES TO PROSECUTE          0.004532
DISTRICT ATTORNEY REFUSES TO PROSECUTE    0.004492
NOT PROSECUTED                            0.004236
JUVENILE CITED                            0.003793
PROSECUTED BY OUTSIDE AGENCY              0.002858
EXCEPTIONAL CLEARANCE                     0.001745
JUVENILE ADMONISHED                       0.001657
JUVENILE DIVERTED                         0.000405
CLEARED-CONTACT JUVENILE FOR MORE INFO    0.000248
PROSECUTED FOR LESSER OFFENSE             0.000058
Name: Resolution, dtype: float64

In [11]:
data['Address'].value_counts(normalize=True)

800 Block of BRYANT ST        0.029969
800 Block of MARKET ST        0.007509
2000 Block of MISSION ST      0.005817
1000 Block of POTRERO AV      0.004620
900 Block of MARKET ST        0.003705
                                ...   
KIRKWOOD AV / MARLIN CT       0.000001
100 Block of COMMERCIAL ST    0.000001
SWEENY ST / CAMBRIDGE ST      0.000001
ISLAIS CREEK ST / 3RD ST      0.000001
RIVERA ST / 34TH AV           0.000001
Name: Address, Length: 23228, dtype: float64

In [12]:
data[['X', 'Y']].describe()

,X,Y
count,875726.000000,875726.000000
mean,-122.422623,37.771032
std,0.030363,0.457497
min,-122.513642,37.707879
25%,-122.432952,37.752427
50%,-122.416446,37.775421
75%,-122.406959,37.784380
max,-120.500000,90.000000


In [13]:
filt = data['Resolution'] == 'NONE'
data.loc[filt, 'Resolution'] = 'UNKNOWN'

In [14]:
data['Resolution'].value_counts(normalize=True)

UNKNOWN                                   0.599642
ARREST, BOOKED                            0.235431
ARREST, CITED                             0.087735
LOCATED                                   0.019286
PSYCHOPATHIC CASE                         0.016591
UNFOUNDED                                 0.010936
JUVENILE BOOKED                           0.006354
COMPLAINANT REFUSES TO PROSECUTE          0.004532
DISTRICT ATTORNEY REFUSES TO PROSECUTE    0.004492
NOT PROSECUTED                            0.004236
JUVENILE CITED                            0.003793
PROSECUTED BY OUTSIDE AGENCY              0.002858
EXCEPTIONAL CLEARANCE                     0.001745
JUVENILE ADMONISHED                       0.001657
JUVENILE DIVERTED                         0.000405
CLEARED-CONTACT JUVENILE FOR MORE INFO    0.000248
PROSECUTED FOR LESSER OFFENSE             0.000058
Name: Resolution, dtype: float64

In [15]:
filt = data['Category'].str.contains('THEFT')
data.loc[filt, 'Category'].unique()

array(['LARCENY/THEFT', 'VEHICLE THEFT'], dtype=object)

In [16]:
test = pd.read_csv("test.csv")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884262 entries, 0 to 884261
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Id          884262 non-null  int64  
 1   Dates       884262 non-null  object 
 2   DayOfWeek   884262 non-null  object 
 3   PdDistrict  884262 non-null  object 
 4   Address     884262 non-null  object 
 5   X           884262 non-null  float64
 6   Y           884262 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 47.2+ MB


In [17]:
data.drop(["Descript", "Resolution"], axis='columns', inplace=True)
data

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541
...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056
878045,2003-01-06 00:01:00,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948
878046,2003-01-06 00:01:00,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266
878047,2003-01-06 00:01:00,VANDALISM,Monday,SOUTHERN,TOWNSEND ST / 2ND ST,-122.390531,37.780607


In [18]:
data.dtypes

Dates          object
Category       object
DayOfWeek      object
PdDistrict     object
Address        object
X             float64
Y             float64
dtype: object

In [19]:
data['Year'] = data['Dates'].apply(lambda x: int(x[0:4]))
data['Month'] = data['Dates'].apply(lambda x: int(x[5:7]))
data['Day'] = data['Dates'].apply(lambda x: int(x[8:10]))

data['Hour'] = data['Dates'].apply(lambda x: int(x[10:13]))
data['Min'] = data['Dates'].apply(lambda x: int(x[14:16]))

data.drop(['Dates'], axis='columns', inplace=True)

data

,Category,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,Hour,Min
0,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23,53
1,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23,53
2,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13,23,33
3,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13,23,30
4,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13,23,30
...,...,...,...,...,...,...,...,...,...,...,...
878044,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003,1,6,0,15
878045,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948,2003,1,6,0,1
878046,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266,2003,1,6,0,1
878047,VANDALISM,Monday,SOUTHERN,TOWNSEND ST / 2ND ST,-122.390531,37.780607,2003,1,6,0,1


In [20]:
def time_of_day(hour):
    if hour >= 0 and hour < 4:
        return 'Late Night'
    elif hour >= 4 and hour < 6:
        return 'Dawn'
    elif hour >= 6 and hour < 12:
        return 'Morning'
    elif hour >= 12 and hour < 16:
        return 'Afternoon'
    elif hour >= 16 and hour < 20:
        return 'Evening'
    elif hour >= 20 and hour <= 23:
        return 'Night'

In [21]:
data['Time_of_Day'] = data['Hour'].map(time_of_day)
data

,Category,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,Hour,Min,Time_of_Day
0,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23,53,Night
1,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23,53,Night
2,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13,23,33,Night
3,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13,23,30,Night
4,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13,23,30,Night
...,...,...,...,...,...,...,...,...,...,...,...,...
878044,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003,1,6,0,15,Late Night
878045,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948,2003,1,6,0,1,Late Night
878046,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266,2003,1,6,0,1,Late Night
878047,VANDALISM,Monday,SOUTHERN,TOWNSEND ST / 2ND ST,-122.390531,37.780607,2003,1,6,0,1,Late Night


In [22]:
data.drop([ 'Hour', 'Min'], axis='columns', inplace=True)
data

,Category,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,Time_of_Day
0,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,Night
1,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,Night
2,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13,Night
3,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13,Night
4,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13,Night
...,...,...,...,...,...,...,...,...,...,...
878044,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003,1,6,Late Night
878045,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948,2003,1,6,Late Night
878046,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266,2003,1,6,Late Night
878047,VANDALISM,Monday,SOUTHERN,TOWNSEND ST / 2ND ST,-122.390531,37.780607,2003,1,6,Late Night


In [23]:
def third_of_month(day):
    if day >= 1 and day < 10:
        return '1st_third'
    elif day >= 10 and day < 20:
        return '2nd_third'
    elif day >= 20 and day <= 31:
        return '3rd_third'

In [24]:
data['Third_of_Month'] = data['Day'].map(third_of_month)
data

,Category,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,Time_of_Day,Third_of_Month
0,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,Night,2nd_third
1,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,Night,2nd_third
2,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13,Night,2nd_third
3,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13,Night,2nd_third
4,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13,Night,2nd_third
...,...,...,...,...,...,...,...,...,...,...,...
878044,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003,1,6,Late Night,1st_third
878045,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948,2003,1,6,Late Night,1st_third
878046,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266,2003,1,6,Late Night,1st_third
878047,VANDALISM,Monday,SOUTHERN,TOWNSEND ST / 2ND ST,-122.390531,37.780607,2003,1,6,Late Night,1st_third


In [25]:
data.drop(['Day'], axis='columns', inplace=True)
data

,Category,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Time_of_Day,Third_of_Month
0,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,Night,2nd_third
1,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,Night,2nd_third
2,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,Night,2nd_third
3,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,Night,2nd_third
4,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,Night,2nd_third
...,...,...,...,...,...,...,...,...,...,...
878044,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003,1,Late Night,1st_third
878045,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948,2003,1,Late Night,1st_third
878046,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266,2003,1,Late Night,1st_third
878047,VANDALISM,Monday,SOUTHERN,TOWNSEND ST / 2ND ST,-122.390531,37.780607,2003,1,Late Night,1st_third


In [26]:
def quarter_of_year(month):
    if month >= 1 and month <= 3:
        return '1st_quarter'
    elif month > 3 and month <= 6:
        return '2nd_quarter'
    elif month > 6 and month <= 9:
        return '3rd_quarter'
    elif month > 9 and month <= 12:
        return '4th_quarter'

In [27]:
data['Quarter_of_Year'] = data['Month'].map(quarter_of_year)
data

,Category,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Time_of_Day,Third_of_Month,Quarter_of_Year
0,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,Night,2nd_third,2nd_quarter
1,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,Night,2nd_third,2nd_quarter
2,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,Night,2nd_third,2nd_quarter
3,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,Night,2nd_third,2nd_quarter
4,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,Night,2nd_third,2nd_quarter
...,...,...,...,...,...,...,...,...,...,...,...
878044,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003,1,Late Night,1st_third,1st_quarter
878045,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948,2003,1,Late Night,1st_third,1st_quarter
878046,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266,2003,1,Late Night,1st_third,1st_quarter
878047,VANDALISM,Monday,SOUTHERN,TOWNSEND ST / 2ND ST,-122.390531,37.780607,2003,1,Late Night,1st_third,1st_quarter


In [28]:
data.drop(['Month'], axis='columns', inplace=True)
data

,Category,DayOfWeek,PdDistrict,Address,X,Y,Year,Time_of_Day,Third_of_Month,Quarter_of_Year
0,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,Night,2nd_third,2nd_quarter
1,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,Night,2nd_third,2nd_quarter
2,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,Night,2nd_third,2nd_quarter
3,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,Night,2nd_third,2nd_quarter
4,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,Night,2nd_third,2nd_quarter
...,...,...,...,...,...,...,...,...,...,...
878044,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003,Late Night,1st_third,1st_quarter
878045,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948,2003,Late Night,1st_third,1st_quarter
878046,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266,2003,Late Night,1st_third,1st_quarter
878047,VANDALISM,Monday,SOUTHERN,TOWNSEND ST / 2ND ST,-122.390531,37.780607,2003,Late Night,1st_third,1st_quarter


In [29]:
def type_of_street_extraction(Address):
    type_of_street= Address.split(' ')
    for index in range(len(type_of_street)):
        cell_value= type_of_street[index]
        if len(cell_value) == 2 and cell_value not in['of', 'OF', 'US', 'LA']:
            #ls_street_types.append(cell_value)
            return cell_value
            break

In [30]:
x= 'hello guys i am tired'
a=type_of_street_extraction(x)
a

'am'

In [31]:
data['street_type'] = data['Address'].map(type_of_street_extraction)

In [32]:
data['street_type'].unique()

array(['ST', 'AV', 'WY', 'BL', 'CT', 'DR', 'TR', 'HY', 'RD', 'PZ', 'PL',
       'LN', 'JR', None, 'CR', 'AL', 'WK', 'EX', 'RW', 'EL', 'DE', 'MC',
       'GG', 'FE', 'MT', 'PT', 'LE', 'SF'], dtype=object)

In [33]:
data.dtypes

Category            object
DayOfWeek           object
PdDistrict          object
Address             object
X                  float64
Y                  float64
Year                 int64
Time_of_Day         object
Third_of_Month      object
Quarter_of_Year     object
street_type         object
dtype: object

In [34]:
frequency_of_category = data[['Category', 'DayOfWeek']].groupby(by=["Category"]).count().sort_values(by='DayOfWeek',
                                                                                                            ascending=False)
frequency_of_category=frequency_of_category.rename(columns={'DayOfWeek': 'frequency'})
frequency_of_category['cum_percent'] = 100 * (frequency_of_category['frequency'].cumsum()/
                                            frequency_of_category['frequency'].sum())
frequency_of_category.reset_index(inplace=True)
frequency_of_category

,Category,frequency,cum_percent
0,LARCENY/THEFT,174320,19.905770
1,OTHER OFFENSES,125960,34.289264
2,NON-CRIMINAL,91915,44.785127
3,ASSAULT,76815,53.556706
4,DRUG/NARCOTIC,53919,59.713769
5,VEHICLE THEFT,53706,65.846509
6,VANDALISM,44581,70.937257
7,WARRANTS,42145,75.749835
8,BURGLARY,36600,79.929224
9,SUSPICIOUS OCC,31394,83.514136


In [35]:
#here, we will filter the data to focus on the categories that contribute for 90% of the outcomes (to follow the pareeto rule),
#so that the classifier has a lower number of labels that result in most of the crimes 
filtered_categories= frequency_of_category[frequency_of_category['cum_percent'] <= 91]
filtered_categories

,Category,frequency,cum_percent
0,LARCENY/THEFT,174320,19.905770
1,OTHER OFFENSES,125960,34.289264
2,NON-CRIMINAL,91915,44.785127
3,ASSAULT,76815,53.556706
4,DRUG/NARCOTIC,53919,59.713769
5,VEHICLE THEFT,53706,65.846509
6,VANDALISM,44581,70.937257
7,WARRANTS,42145,75.749835
8,BURGLARY,36600,79.929224
9,SUSPICIOUS OCC,31394,83.514136


In [36]:
#we will filter now the training data using the categories mentioned above only;
data = data[data.Category.isin(filtered_categories['Category'])]
data.reset_index(inplace=True)
data

,index,Category,DayOfWeek,PdDistrict,Address,X,Y,Year,Time_of_Day,Third_of_Month,Quarter_of_Year,street_type
0,0,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,Night,2nd_third,2nd_quarter,ST
1,1,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,Night,2nd_third,2nd_quarter,ST
2,2,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,Night,2nd_third,2nd_quarter,AV
3,3,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,Night,2nd_third,2nd_quarter,ST
4,4,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,Night,2nd_third,2nd_quarter,ST
...,...,...,...,...,...,...,...,...,...,...,...,...
796644,878043,OTHER OFFENSES,Monday,BAYVIEW,1500 Block of SHAFTER AV,-122.389769,37.730564,2003,Late Night,1st_third,1st_quarter,AV
796645,878044,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003,Late Night,1st_third,1st_quarter,ST
796646,878045,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948,2003,Late Night,1st_third,1st_quarter,ST
796647,878046,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266,2003,Late Night,1st_third,1st_quarter,ST


In [37]:
le = LabelEncoder()
data['Category_Labels'] = le.fit_transform(data['Category'])
data['DayOfWeek_Labels'] = le.fit_transform(data['DayOfWeek'])
data['PdDistrict_Labels'] = le.fit_transform(data['PdDistrict'])
data['Time_of_Day_Labels'] = le.fit_transform(data['Time_of_Day'])
data['Third_of_Month_Labels'] = le.fit_transform(data['Third_of_Month'])
data['Quarter_of_Year_Labels'] = le.fit_transform(data['Quarter_of_Year'])
data['street_type_Labels'] = le.fit_transform(data['street_type'])
data

C:\Users\Hesham\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Hesham\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Hesham\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,index,Category,DayOfWeek,PdDistrict,Address,X,Y,Year,Time_of_Day,Third_of_Month,Quarter_of_Year,street_type,Category_Labels,DayOfWeek_Labels,PdDistrict_Labels,Time_of_Day_Labels,Third_of_Month_Labels,Quarter_of_Year_Labels,street_type_Labels
0,0,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,Night,2nd_third,2nd_quarter,ST,12,6,4,5,1,1,23
1,1,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,Night,2nd_third,2nd_quarter,ST,7,6,4,5,1,1,23
2,2,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,Night,2nd_third,2nd_quarter,AV,7,6,4,5,1,1,1
3,3,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,Night,2nd_third,2nd_quarter,ST,4,6,4,5,1,1,23
4,4,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,Night,2nd_third,2nd_quarter,ST,4,6,5,5,1,1,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796644,878043,OTHER OFFENSES,Monday,BAYVIEW,1500 Block of SHAFTER AV,-122.389769,37.730564,2003,Late Night,1st_third,1st_quarter,AV,7,1,0,3,0,0,1
796645,878044,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003,Late Night,1st_third,1st_quarter,ST,8,1,8,3,0,0,23
796646,878045,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948,2003,Late Night,1st_third,1st_quarter,ST,4,1,2,3,0,0,23
796647,878046,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266,2003,Late Night,1st_third,1st_quarter,ST,4,1,7,3,0,0,23


In [38]:
data=data.drop(columns=['Category', 'DayOfWeek','PdDistrict','Address', 'Time_of_Day',
                        'Third_of_Month', 'Quarter_of_Year', 'street_type', 'index'])
data

,X,Y,Year,Category_Labels,DayOfWeek_Labels,PdDistrict_Labels,Time_of_Day_Labels,Third_of_Month_Labels,Quarter_of_Year_Labels,street_type_Labels
0,-122.425892,37.774599,2015,12,6,4,5,1,1,23
1,-122.425892,37.774599,2015,7,6,4,5,1,1,23
2,-122.424363,37.800414,2015,7,6,4,5,1,1,1
3,-122.426995,37.800873,2015,4,6,4,5,1,1,23
4,-122.438738,37.771541,2015,4,6,5,5,1,1,23
...,...,...,...,...,...,...,...,...,...,...
796644,-122.389769,37.730564,2003,7,1,0,3,0,0,1
796645,-122.459033,37.714056,2003,8,1,8,3,0,0,23
796646,-122.447364,37.731948,2003,4,1,2,3,0,0,23
796647,-122.403390,37.780266,2003,4,1,7,3,0,0,23


In [39]:
data['Category_Labels'].unique()

array([12,  7,  4, 11, 10,  6,  8,  0,  1,  9,  2,  5,  3])